In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torchaudio
from torchaudio import datasets, transforms
import pandas as pd
import librosa
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
#import soundata

In [2]:
#this code should process the data from UrbanSound8k

#the sound data is split into 10 folds,, 
#the data needs a 10-fold cross validation ,, train on folds 1-9 and then test on 10
#then train on every fold except 9 and then test on 9 
#etc etc

batch_size = 20

#the nosisy speech data base has seperate folders for clean and noisy data already
#we are goin gto use those,,
#turn them into spectrograms
#and then start training and testing on the linear model

#urbansound_metadata = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')





In [3]:
#this code should prep data from NoisyDataset

#load dataset
def load_audio_files(filepath):
    files = glob(os.path.join(filepath, '*.wav'))
    audio_data = []
    for file in files:
        y, sr = librosa.load(file, sr=None)
        audio_data.append((y, sr))
    return audio_data


def conv_spectrograms(files, n_fft=1024, hop_length=512):
    spectrograms = []
    for y in files:
        s = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
        db = librosa.amplitude_to_db(np.abs(s), ref=np.max)
        spectrograms.append(db)
    return np.array(spectrograms)


clean_traindata = "./clean_trainset_28spk_wav"
noisy_traindata = "./noisy_trainset_28spk_wav"

clean_testdata = "./clean_testset_wav"
noisy_testdata = "./noisy_testset_wav"


clean_spectrograms = conv_spectrograms(clean_traindata)
noisy_spectrograms = conv_spectrograms(noisy_traindata)


#create a noisy dataset


OSError: cannot load library 'libsndfile.dll': error 0x7e

In [ ]:
#Building a basic denoising model
#linear model

class DenoisingLinearModel(nn.Module):
    def __init__(self, input, output):
        super(DenoisingLinearModel, self).__init__()
        self.linear = nn.Linear(input, output) ##one layer to 
        #self.linear

    def forward(self, x):
        x = self.linear(x)
        return x
    
#this is a basic linear regression model

#if this does not give the correct outputs,
#switch to CNN model

def create_data(dataset, batch_size):
    load_data = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    #can shuffle bc its the noisy speech database
    return load_data



train_data = create_data(list(zip(noisy_spectrograms, clean_spectrograms)))
#returns (noisy,clean)


#object of model:
ln_model = DenoisingLinearModel()

In [ ]:
#loss function and optimizer

#mean squared error loss function 
loss_fn = nn.MSELoss()
optimizer =torch.optim.sgd(ln_model, lr = 0.01)

In [ ]:
#train the model

correct = 0
total = 0

num_epochs = 10
for epoch in range(num_epochs):
    for data in train_data:
        noisy, clean = data
        output = DenoisingLinearModel(noisy)
        _, predicted = torch.max(output.data, 1)
        total += clean.size(0)
        correct += (predicted == clean).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
#second experiment

In [ ]:
#third experiment